#### Step 1 : Import Required Libs

In [22]:
## Import required Libs
import boto3
import json
from botocore.exceptions import BotoCoreError, ClientError


#### Step 2 : Set AWS Region

In [23]:
## Set AWS Region
try:
    # Set AWS Region
    region = "us-east-1"  # Note the correct region code "us-east-1"
    
    # The Bedrock runtime must be initialized with your AWS credentials/profile and region to authenticate you 
    # and direct requests to the right foundation models or agent.
    bedrock_runtime = boto3.client("bedrock-runtime", region_name=region)
    s3 = boto3.client("s3", region_name=region)
    
    print("Project Environment is setup successfully")
    
except (BotoCoreError, ClientError) as error:
    print(f"An error occurred while setting up AWS clients: {error}")


Project Environment is setup successfully


#### Step-3 : Setup RAG Function

###### 1. Take user query
###### 2. Retrieve relevant information from our knowledge base
###### 3. Generate a response using a foundation model

In [24]:
##--------------------------------------------------------------------------------------------------------------------------------------------- 
## 1] RAG Model - Amazon Titan text foundation model available via Amazon Bedrock.
##  This model is a large language model (LLM) optimized for a wide range of advanced, general-purpose language
##  tasks such as open-ended text generation, conversational chat, and also supports Retrieval Augmented Generation (RAG) workflows.
##  ---
## 2] knowledge_base_id identifies the knowledge base to query.This is a unique identifier (typically a string or an Amazon Resource Name, ARN)
##  that specifies which knowledge base your application or model should access during information retrieval.
##  In the context of Amazon Bedrock: A knowledge base is a structured repository of information—often created by ingesting private documents,
##  databases, or other content. Amazon Bedrock uses these knowledge bases to support Retrieval-Augmented Generation (RAG) workflows.
##  The knowledge_base_id is the unique ID assigned to a specific knowledge base within Amazon Bedrock. When issuing a query,
##  this ID tells Bedrock which knowledge base to search for relevant information.
##---------------------------------------------------------------------------------------------------------------------------------------------
##
model_id = "amazon.titan-text-express-v1"
def generate_rag_response(query,knowledge_base_id, model_id = model_id):
    """ Generate Response using RAG Approach
        Args :
         1] query (str) : The user's question
         2] knowledge_base_id (str) : ID of the knowledge base to query
    """
    try: 
        """ Step-1 : Retrieve information from the knowledge base
        # Note : In a full implemenation, we would call the Bedrock Knowledge Base API here
        # For this example, we will simulate the retrieval with a placeholder
        # bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name = region)
        # kb_response = bedrock_agent_runtime.retrieve(
           knowledgeBaseId=knowledge_base_id,
           retrivalQuery ={'text' : query},
           numberOfResults = 3)
       """
       ## For simplified example:
        retrieved_info =""" Employee Handbook Section 3.2 : Pait Time Off
         All Employees accrue PTO at a rate of 1.5 days per month (18 days per year).
         PTO requests must be submitted at least two weeks in advance through the HR portal
         Unused PTO can be carried over to the following year, up to a maximum of 5 days
         For further queries on PTO, please contact hr@example.com

         Remote work policy for full time employees only - Full time employees are eligible to work remotely 
         up to 2 days per week and max of 10 days per month under unavoidable circumstances only when manager 
         has approved additional remote work days. Under normal circumstances, full time employees are expected to work from office for 3 days a week.
         For consitency, co-ordination and efficiency purpose, all employees are expected to work from
         office from Monday to Wednesday.
                                          """
        
        # Step 2 - Construct the prompt for the model using retrieved information and query
        prompt = f"""
          You are an HR assistant for our company.Use only following information to answer the query.
          If you do not know the answer based on this information then say No - don't make up an answer

          RETRIEVED INFORMATION :
          {retrieved_info}

          USER QUESTION:
          {query}
            """
    
        # Step 3 - Construct the payload
        # Generate a response using the specified model
        if 'claude' in model_id.lower():
            payload = {
               "anthropic_version": "bedrock-2023-05-31", # Specify the version of the Anthropic model to use
               "max_tokens": 512, # How many words (max tokens) the AI can respond with (up to 512).
               "temperature" : 0.5, #How “creative” or random the answers should be (temperature 0.5 means moderately creative).
               "messages" :[
                    {
                         "role": "user",# The role of the user in the conversation
                         "content": prompt # The actual text of the user message, which includes the retrieved information and the user's question
                     
                  }
                  
               ]
            }
        else:
            # Generic format for other models.
            payload = {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": 256,
                    "temperature": 0.9
                }
            }
        # Step 4 - Call the model   
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(payload), # Convert the payload to JSON format
            contentType="application/json", # Specify the content type as JSON
            accept="application/json" # Specify the response format as JSON
        )
        # Step 5 - Parse the response based on the model type.
        response_body = json.loads(response['body'].read().decode('utf-8')) # Read and decode the response body
        ###       
        if 'claude' in model_id.lower():
            generated_text = response_body["content"][0]["text"]
        else:
            generated_text = response_body["results"][0]["outputText"]
        
        return generated_text
    
    except Exception as e:
        return f"An error occurred while generating the response:{str (e)}"


#### Step 4:  Test RAG Application

In [25]:
# Define a sample knowledge base ID - This would be a real ID in a production scenario
sample_kb_id = "sample hr knowledge base id"  # Replace with your actual knowledge base ID
# Test query
# test_query = "How many PTO days do full-time employees get per year?"
#test_query = "Is current remote work policy is applicable for full time employees only? What about part time employees?"
test_query = " What is the role of my manager ?"
# test_query = "Why all employees are expected to work from office from Monday to Wednesday?"  # Uncomment to use this query
# # Call the function to generate a response
response = generate_rag_response(test_query, sample_kb_id)
print("\n User Query:", test_query)
print("\n Response from RAG Model:", response)
# --- End  ---




 User Query:  What is the role of my manager ?

 Response from RAG Model: 
The manager is expected to approve additional remote work days when circumstances arise, and to coordinate and ensure efficiency in the workplace


In [26]:
# Test with a question that should be answerable from our knowledge base
test_query_2 = "What's the process for requesting time off?"
response_2 = generate_rag_response(test_query_2, sample_kb_id)

# Test with a question that might not be in our knowledge base
test_query_3 = "What's the company policy on remote work?"
response_3 = generate_rag_response(test_query_3, sample_kb_id)

print("\nQuestion about time off process:")
print(response_2)

print("\nQuestion about remote work policy (not in our example knowledge base):")
print(response_3)



Question about time off process:

No.

Question about remote work policy (not in our example knowledge base):

The company policy on remote work is that full time employees are eligible to work remotely up to 2 days per week and max of 10 days per month under unavoidable circumstances only when manager has approved additional remote work days.


In [27]:
# Set up the Knowledge Base Using Vector Database
# Step 1 - Create S3 bucket. Create markdown file containing HR Policies and store in AWS s3

import boto3

# Initialize boto3 S3 client
s3 = boto3.client('s3')

# Generate a unique bucket name using your account ID
account_id = boto3.client('sts').get_caller_identity()['Account']
bucket_name = f"hr-knowledge-base-v02-{account_id}"

# Check if the bucket already exists
def bucket_exists(bucket_name):
    response = s3.list_buckets()
    buckets = [bucket['Name'] for bucket in response['Buckets']]
    return bucket_name in buckets

# Create the bucket if it doesn't exist
if not bucket_exists(bucket_name):
    try:
        s3.create_bucket(Bucket=bucket_name)
        print(f"Created bucket: {bucket_name}")
    except Exception as e:
        print(f"Error creating bucket: {str(e)}")
else:
    print(f"Bucket {bucket_name} already exists")

Bucket hr-knowledge-base-v02-257269733378 already exists


In [28]:
# Create a sample HR policy markdown file
hr_policy_v03 = """ ABCSoftware - Human Resources (HR) Policy-V1.3
Preamble
The Human Resources (HR) Policy of [Organization Name] defines the guiding principles and procedures for effective management of our people. This policy ensures fair, consistent, and legally compliant practices to cultivate a productive, respectful, and welcoming workplace. We recognize our employees as our greatest asset and are committed to creating an environment that attracts, develops, and retains talent while supporting our mission, values, and strategic objectives.

1. Recruitment and Selection
The organization is committed to a transparent, merit-based recruitment process that welcomes qualified candidates aligned with our values and job requirements. This policy applies to all forms of employment, whether permanent, contractual, part-time, or temporary. HR will work with department heads annually to assess workforce needs and maintain current job descriptions. Vacancies are advertised both internally and externally to promote equal opportunity, and the selection process includes structured interviews, competency assessments, and background checks. All recruitment shall comply with non-discrimination laws to avoid any bias regarding age, gender, race, religion, or disability. Offer letters are provided only after all necessary approvals.

2. Onboarding and Induction
We strive to integrate new hires smoothly into our culture and processes. Upon joining, HR provides offer confirmation, joining instructions, and a list of required documents. New employees participate in an orientation program introducing our history, values, structure, key policies, IT systems, and safety procedures. All new employees serve a probationary period of three to six months, with formal reviews at 30, 60, and 90 days. A workplace buddy is assigned for the first month to assist in acclimatization.

3. Employee Classification and Contracts
Employees are classified into permanent, fixed-term contract, part-time, and internships or traineeships. Each employment contract clearly states the role, pay, benefits, working hours, probation period, and termination clauses, in compliance with applicable labor laws.

4. Compensation and Benefits
Salaries are defined by job grade, market benchmarks, and internal equity and are reviewed annually. Employees receive health insurance for themselves and eligible dependents, retirement benefits according to statutory requirements, paid leave entitlements (annual, sick, maternity/paternity, and compassionate), and access to an Employee Assistance Program for mental wellbeing. Salaries are processed monthly with electronic payslips, and statutory deductions are made as per legal requirements.

5. Working Hours, Attendance, and Leave
The standard workweek is [e.g., 40 hours, Monday through Friday, 9 AM to 6 PM], with flexible work arrangements available with managerial approval. Attendance is tracked electronically; staff must clock in and out. Uninformed absences beyond three days may lead to disciplinary action. Leave entitlements include a set number of annual and sick days, with additional leave for maternity/paternity in accordance with the law. Unpaid leave is possible with approval.

6. Performance Management
We aim to fairly evaluate performance, aligning individual goals with the organization's objectives. Annual goals are set collaboratively and formally reviewed mid-year, with an annual appraisal focusing on key performance indicators, competencies, and behaviors. Performance improvement plans are available for underperforming staff, with defined timelines and support.

7. Learning and Development
The organization invests in employee growth through ongoing training and development opportunities. Training needs are assessed each year, and both internal and external programs are provided. High-potential employees benefit from career development plans, and tuition assistance is available for approved studies.

8. Employee Conduct and Discipline
Employees must demonstrate integrity, respect, and professionalism at all times. Disciplinary procedures follow a progressive process from verbal to written to final warning, then termination as appropriate, with immediate dismissal possible for gross misconduct such as fraud, violence, or harassment. There is zero tolerance for harassment, bullying, or discrimination; complaints will be investigated confidentially.

9. Health, Safety, and Welfare
[Organization Name] complies with all Occupational Health and Safety laws. Annual risk assessments are conducted and first aid facilities and emergency procedures are maintained. Employees are encouraged to participate in wellness programs, including annual health check-ups, fitness subsidies, and mental health initiatives.

10. Exit and Termination
When an employee voluntarily resigns, the required notice period (typically one month) must be served, and an exit interview is conducted. Termination by the employer may occur due to performance issues, redundancy, or misconduct; in these cases, notice or payment in lieu is provided per contract. Final settlements cover the return of company property and payment of remaining salary, benefits, and any applicable leave encashment.

Implementation and Review
This HR policy will be reviewed annually by the HR department. Any changes must be approved by both the HR Director and the CEO to ensure ongoing relevance and legal compliance
"""

In [29]:
## Save the HR policy content to a markdown file in S3
with open('hr_policy_v03.md', 'w', encoding='utf-8') as file:
    file.write(hr_policy_v03)
# Upload the markdown file to the S3 bucket
try:    
    s3.upload_file('hr_policy_v03.md', bucket_name, 'hr_policy_v03.md')
    print(f"HR policy file uploaded to S3 bucket {bucket_name} successfully.")  
except Exception as e:
    print(f"Error uploading HR policy file to S3: {str(e)}")    
    

HR policy file uploaded to S3 bucket hr-knowledge-base-v02-257269733378 successfully.


In [30]:
import json
import boto3

# model_id = "amazon.titan-text-express-v1"  # Default
# model_id = "amazon.titan-embed-image-v1"  # Default
#model_id = "amazon.titan-text-express-v1"
model_id = "amazon.titan-embed-text-v1"
# model_id = "amazon.titan-text-premier-v1:0"  # Default
# model_id = "anthropic.claude-v2:1"  # Default
region = "us-east-1"  # Change this to your AWS region

def query_hr_knowledge_base(user_query, knowledge_base_id, model_id=model_id, region=region):
    """
    Query HR Knowledge Base and generate a response using a RAG Model.
    Args:
        user_query (str): The user's question.
        knowledge_base_id (str): ID of the knowledge base to query.
        model_id (str): The model ID to use.
        region (str): AWS region where Bedrock is deployed.
    """
    # bedrock-runtime client: For basic model invocation and runtime interactions with foundation models on Bedrock.
    # bedrock-agent-runtime client: For advanced agent-driven interactions, supporting knowledge bases, 
    # session context, and retrieval-augmented generation workflows.
    bedrock_runtime = boto3.client("bedrock-runtime", region_name=region)
    bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=region)

    try:
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=knowledge_base_id,
            retrievalQuery={'text': user_query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 5  # Top 5 relevant results
                }
            }
        )

        # Extract retrieved information
        retrieved_passages = []
        for result in retrieve_response.get('retrievedResults', []):
            content = result.get('content', {}).get('text', '')
            source = "Unknown source"
            location = result.get('location', {})

            if location:
                if 's3location' in location:
                    source = location.get('s3location', {}).get('uri', source)
                elif 'type' in location:
                    source = location.get('type', source)

            score = result.get('score', 0)
            retrieved_passages.append({
                'content': content,
                'source': source,
                'relevance_score': score
            })

        # Prepare context
        context = "\n\n".join([f"Passage: {p['content']}" for p in retrieved_passages])
        if not context:
            context = "No relevant information found in the knowledge base."

        # Construct prompt
        prompt = f"""
        You are HR Manager who is helping employees to easily understand policies.  Use only the following information to answer the query.
        If you do not know the answer based on this information then say:
        'No I do not have definitive answer for this as of now' — don't make up an answer.

        Context:
        {context}

        Question: {user_query}

        Answer:
        """

        # Select payload based on model type
        if "anthropic" in model_id.lower():
            print("Inside Anthropic Model")
            payload = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 512,
                "temperature": 0.5,
                "messages": [{"role": "user", "content": prompt}]
            }
        # elif "amazon.titan" in model_id.lower():
        #     print("Inside Amazon Titan Model")
        #     payload = {
        #         "inputText": prompt,
        #         "textGenerationConfig": {
        #             "maxTokenCount": 512,
        #             "temperature": 0.6,
        #             "topP": 0.9
        #         }
        #     }
        elif "amazon.titan" in model_id.lower():
            print("Inside Amazon Titan Model")

            if "embed" in model_id.lower():
        # Embedding model payload (no textGenerationConfig)
                 payload = {
                "inputText": prompt
                }
            else:
             # Text generation model payload
                 payload = {
                   "inputText": prompt,
                   "textGenerationConfig": {
                   "maxTokenCount": 1000,
                   "temperature": 0.9,
                   "topP": 0.7
            }
        }
    #----###
        elif "meta.llama" in model_id.lower():
            print("Inside Meta Llama Model")
            payload = {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": 512,
                    "temperature": 0.7,
                    "topP": 0.9
                }
            }
        else:
            raise ValueError(f"Unsupported model ID: {model_id}. Please use a supported model ID for RAG workflows.")

        # Call the model
        invoke_response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(payload),
            contentType="application/json",
            accept="application/json"
        )

        # Parse the response
        response_body = json.loads(invoke_response['body'].read().decode('utf-8'))

        if "anthropic" in model_id.lower():
            generated_answer = response_body.get('content', [{}])[0].get("text", "")
        elif "amazon.titan" in model_id.lower():
            generated_answer = response_body.get('results', [{}])[0].get("outputText", '')
        elif "meta.llama" in model_id.lower():
            generated_answer = response_body.get('generation', '')
        else:
            generated_answer = "Error: Could not parse response from the model."

        # Final return
        return {
            "query": user_query,
            "retrieved_passages": retrieved_passages,
            "generated_answer": generated_answer,
            "model_used": model_id
        }
    # Titan Embeddings G1 - Textv1.2
    except Exception as e:
        return {
            "error": str(e),
            "query": user_query,
            "knowledge_base_id": knowledge_base_id,
            "retrieved_passages": [],
            "model_id": model_id
        }


In [31]:
##Retrievel Example
# Define a sample knowledge base ID - This would be a real ID in a production scenario
sample_kb_id = "GIVL72SHRJ"  # Replace with your actual knowledge base ID
#sample_kb_id = "ZHJZPXTKEV"
# sample_kb_id = "KU4HIUJJNS"  
# sample_kb_id = "KU4HIUJJNS"
print(f"Model Id is : {model_id}")
# Test with a question that should be answerable from our knowledge base
# query = "What type of wellness programs are run by company ?"
# query = 'Who conducts annual workforce planning ?'
# query = "Who can avail Remote working benefit?"
# query = "When medical certificate is required ?"
query = "What is the Disciplinary Process of the company?"
# query = "On what basis employee's compensation & benefits are revised or determined ?"
# query = "What is the base of Annual Appraisal ?"
# query = "What is the health and safety policy of the company."
# query = "How frequently salary is paid ?"  # Uncomment to use this query
result = query_hr_knowledge_base(query, sample_kb_id, model_id)
if result is not None:
    print("\n" + "=="*30)
    print(f"Query: {result.get('query', '')}")
    if result.get('retrieved_passages'):
        print(f"\nRetrieved Information:\n{result['retrieved_passages'][0]['content']}")
    else:
        print("\nNo retrieved information found.")
    if 'generated_answer' in result:
        print(f"\nGenerated Answer:\n {result['generated_answer']}")
    elif 'error' in result:
        print(f"\nError:\n {result['error']}")
    else:
        print("\nNo answer generated.")
## End of my experiment with RAG Model - I am tired now and it is 5.18 AM in the morning. I will continue later. ##



Model Id is : amazon.titan-embed-text-v1
Inside Amazon Titan Model

Query: What is the Disciplinary Process of the company?

No retrieved information found.

Generated Answer:
 
